In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../../")
print(os.getcwd())

C:\Users\Milosz\Desktop\python\thesis-recsys


In [24]:
import pandas as pd
import numpy as np
import functools
import operator
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import namedtuple

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

torch.set_printoptions(precision=2, sci_mode=False)
torch.manual_seed(0)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
import pickle
with open("data/steam/data.pkl", "rb") as f:
    data = pickle.load(f)

In [5]:
user_col = 'user_id'
item_col = 'app_id'
train_set = data['train_set'][[user_col, item_col]].values.T
supervision_set = data['supervision_set'][[user_col, item_col]].values.T
valid_set = data['valid_set'][[user_col, item_col]].values.T
user_attr = data['user_attr']
item_attr = data['item_attr']

train_set = np.concatenate([train_set, supervision_set], axis=1)

In [ ]:
# scaler_user = StandardScaler()
# user_attr_preprocess = scaler_user.fit_transform(user_attr)

# scaler_item = StandardScaler()
# item_attr_preprocess = np.copy(item_attr)
# item_attr_preprocess[:, 435:] = scaler_item.fit_transform(item_attr[:, 435:])

In [28]:
item_attr[10]

array([1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 96.0, 68327.0,
       9.99, 9.99, 0.0,
       list([130, 311, 1, 228, 322, 72, 16, 317, 139, 24, 320, 344, 36, 222, 22, 155, 20, 297, 108, 282])],
      dtype=object)

In [29]:
scheme = {
    "user_id": {
        "emb_dim": 8,
        "num_emb": user_attr.shape[0],
        "max_len": 1,
        "type": 'sparse'
    },
    "item_id": {
        "emb_dim": 8,
        "num_emb": item_attr.shape[0],
        "max_len": 1,
        "type": 'sparse'
    },
    "tags": {
        "emb_dim": 8,
        "num_emb": 425,
        "max_len": 20,
        "type": 'varlen'
    },
    "OS": {
        "emb_dim": 8,
        "num_emb": 3,
        "max_len": 3,
        "type": 'varlen'
    },
    "AvgCatRating": {
        "emb_dim": 8,
        "num_emb": 6,
        "max_len": 1,
        "type": 'sparse'
    }
}

class SparseFeat(namedtuple('SparseFeat', ['name', 'emb_dim', 'num_emb', "max_len", 'index'])):
    def __new__(cls, name, emb_dim, num_emb, max_len, index):
        return super(SparseFeat, cls).__new__(cls, name, emb_dim, num_emb, max_len, index)
    
class DenseFeat(namedtuple('DenseFeat', ['name', 'index'])):
    def __new__(cls, name, index):
        return super(DenseFeat, cls).__new__(cls, name, index)    
    
# class VarLenSparseFeat(namedtuple('VarLenSparseFeat', ['name', 'emb_dim', 'num_emb', "max_len", 'index'])):
#     def __new__(cls, name, emb_dim, num_emb, max_len, index):
#         return super(VarLenSparseFeat, cls).__new__(cls, name, emb_dim, num_emb, max_len, index)

class FeatureVault:
    def __init__(self, scheme):
        self.features = {'sparse': [], 'dense': []}
        
        start = 0
        for feat_name, feat in scheme.items():
            if feat['type'] == 'sparse':
                self.features['sparse'].append(SparseFeat(feat_name, feat['emb_dim'], feat['num_emb'], feat['max_len'], (start, start+feat['max_len'])))
                start += feat['max_len'],
            elif feat['type'] == 'dense':
                self.features['dense'].append(DenseFeat(feat_name, (start, start+1)))
                start += 1
        
    def input_len(self):
        sparse_len = sum([f.emb_dim for f in self.features['sparse']])
        dense_len = len(self.features['dense'])
        return sparse_len + dense_len
    
    def emb_dim(self):
        return 8
    
    def num_emb(self):
        return sum([f.num_emb for f in self.features['sparse']])

In [ ]:
class DeepFMDataset(Dataset):
    def __init__(self, edge_index, user_attr, item_attr):
        self.edge_index = edge_index
        self.user_attr = user_attr
        self.item_attr = item_attr
        
        self.users = edge_index[0]
        self.items = edge_index[1]
    
    def __len__(self):
        return self.edge_index.shape[1]
    
    def __getitem__(self, idx):
        u_id = self.users[idx]
        i_id = self.items[idx]
        
        u_attr = self.user_attr[u_id]
        i_attr = self.item_attr[i_id]
        
        sparse_features = self._get_sparse_features(u_id, i_id, u_attr, i_attr)
        dense_features = self._get_dense_features(u_attr, i_attr)
        
        return sparse_features, dense_features
    
    def _get_sparse_features(self, u_id, i_id, u_attr, i_attr):
        return [u_id, i_id, 
    
    def _get_dense_features(self, u_attr, i_attr):
        return 

In [21]:
dataset = DeepFMDataset(train_set, user_attr, item_attr)

NameError: name 'DeepFMDataset' is not defined

In [22]:
dataset

NameError: name 'dataset' is not defined

In [ ]:
class MF(nn.Module):
    def __init__(self, n_users, n_items, emb_size):
        super().__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.u = nn.Embedding(n_users, emb_size)
        self.i = nn.Embedding(n_items, emb_size)
        
    def forward(self, ux, ix):
        return torch.sum(self.u(ux) * self.i(ix), dim=1)
        #return torch.sum((self.u(ux) * self.i(ix)).squeeze(1), dim=1)

In [ ]:
train_dataset = DeepFMDataset(train_set, user_attr, item_attr_preprocess)

In [ ]:
features = FeatureVault(scheme)

In [ ]:
features.features

In [ ]:
class CrossProductNet(nn.Module):
    def forward(self, emb_x):
        cross = emb_x @ emb_x.permute(1, 0)
        triangle_lower_cross = torch.tril(cross, diagonal=-1)
        return torch.sum(triangle_lower_cross)

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout=0.1, batch_norm=True):
        super(MLP, self).__init__()
        
        _layers = []
        dims = [input_dim] + hidden_dim + [1]
        for in_dim, out_dim in zip(dims, dims[1:]):
            _layers.append(nn.Linear(in_dim, out_dim))
            if batch_norm:
                _layers.append(nn.BatchNorm1d(out_dim))
            _layers.append(nn.ReLU())
            _layers.append(nn.Dropout(p=dropout))

        self.layers = nn.ModuleList(_layers)
        
    def forward(self, emb_x):
        return self.layers(emb_x)
    
# class EmbeddingNet(nn.Module):
#     def __init__(self, features):
#         super(EmbeddingNet, self).__init__()
#         _embeddings = {feat.name: nn.Embedding(feat.emb_values, feat.emb_dim) for feat in features}
#         self.features = features
#         self.embeddings = nn.ModuleDict(_embeddings)
        
#     def forward(self, x):
#         emb = []
#         for name, embed_matrix in self.embeddings.items():
#             if isinstance(x[name], list):
#                 emb.append(torch.sum(embed_matrix(torch.tensor(x[name])), axis=0))
#             else:
#                 emb.append(embed_matrix(torch.tensor(x[name])))
#         return emb

class EmbeddingNet(nn.Module):
    def __init__(self, sparse_feat, varlen_feat, emb_dim):
        super(EmbeddingNet, self).__init__()
        self.sparse_emb = nn.Embedding(sparse_feat, emb_dim)
        self.varlen_sparse_emb = nn.Embedding(varlen_feat, emb_dim)
        
        self.offsets_sparse = np.array((0, *np.cumsum(sparse_feat)[:-1]))
        self.offsets_varlen = np.array((0, *np.cumsum(varlen_feat)[:-1]))
        
    def forward(self, x):
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return self.embedding(x)       
    
class DeepFM(nn.Module):
    def __init__(self, features, hidden_dim):
        super(DeepFM, self).__init__()
        self.features = features
        
        sparse_num_emb, varlen_num_emb = self.features.num_emb()
        input_dim = self.features.input_len()
        emb_dim = self.features.emb_dim()
        
        self.V = EmbeddingNet(sparse_num_emb, varlen_num_emb, emb_dim)
        self.fm = CrossProductNet()
        self.dnn = MLP(input_dim, hidden_dim)
        
    def forward(self, x):
        x_sparse, x_varlen, x_dense = filter_input_types(x, self.features)
        
        x_emb = self.V(x_sparse_emb)
        x = self.fm(x_emb) + self.dnn(x_emb)
        return x

In [ ]:
model = DeepFM(features, hidden_dim=[128,64])

In [ ]:
torch.tensor([[1,2,3], [4,5,6], [7,8,9]])[[1,2]]

In [ ]:
model

In [ ]:
features.features

In [ ]:
train_dataset[9432]

In [ ]:
filter_input_types(train_dataset[9432], features)

In [ ]:
def filter_input_types(x, features):
    feats_tensors = []
    for feat_type, feat_list in features.features.items():
        lst = []
        for feat in feat_list:
            lst.append(x[feat.index[0]:feat.index[1]])
        feats_tensors.append(torch.cat(lst) if lst else [])
    return feats_tensors

In [ ]:
e = EmbeddingNet([f.num_emb for f in features], 8)

In [ ]:
np.array((0, *np.cumsum([f.num_emb for f in features])[:-1]))

In [ ]:
[f.num_emb for f in features]

In [ ]:
e.offsets

In [ ]:
torch.arange(20).new_tensor([0, 5, 10]).unsqueeze(0)

In [ ]:
x_input = {
    "user_id": 5,
    "item_id": 20,
    "tags": [0, 5, 8, 100],
    "OS": [0, 1],
    "AvgCatRating": 3 
}
x_input_tensor = torch.tensor([5, 
                               20, 
                               0, 5, 8, 100,
                               0, 1,
                               3
                              ])

In [ ]:
e(x_input)

In [ ]:
def approx_negative_sampling(u_id, i_id, n_samples, bs):
    ind = torch.arange(bs).repeat(n_samples)
    perm =  torch.randperm(ind.shape[0])
    random_indices = ind[perm]
    i_id_neg = i_id[random_indices]
    
    u_id = u_id.repeat(n_samples+1)
    i_id = torch.cat([i_id, i_id_neg])
    y_true = torch.cat([torch.ones(bs).to(device),
                       torch.zeros(bs*n_samples).to(device)])
    
    return u_id, i_id, y_true

In [ ]:
def train(n_epochs, print_loss=500):
    model.train()
    batch_size = train_loader.batch_size
    
    for epoch in range(n_epochs):
        running_loss = 0.
        preds, ground_truths = [], []
        batch_print_loss = 0.
        for i_batch, batch in enumerate(tqdm(train_loader)):
            batch = [i.to(device) for i in batch]
            u_id, i_id, u_attr, i_attr = batch
 
            u_id, i_id, y_true = approx_negative_sampling(u_id, i_id, n_samples=2, bs=batch_size)
            
            y_pred = model(u_id, i_id)
            loss = criterion(y_pred, y_true)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            preds.append(y_pred)
            ground_truths.append(y_true)
            running_loss += loss.item()
            
            if not ((i_batch+1) % print_loss):
                pred = torch.cat(preds, dim=0).detach().sigmoid().cpu().numpy()
                ground_truth = torch.cat(ground_truths, dim=0).detach().cpu().numpy()
                last_loss = running_loss / print_loss
                
                train_roc_auc = roc_auc_score(ground_truth, pred)
                test_loss, test_roc_auc = test()
                
                preds, ground_truths = [], []
                running_loss = 0.
                
                print(f"batch <{i_batch}>\ntrain_loss: {last_loss} - train_roc_auc: {train_roc_auc}\ntest_loss: {test_loss} - test_roc_auc: {test_roc_auc}\n")
        print(f"Epoch: {epoch}, Loss: {running_loss / len(train_loader):.4f}")

In [ ]:
@torch.no_grad()
def test():
    model.eval()
    batch_size = val_loader.batch_size
    
    running_loss = 0.
    preds, ground_truths = [], []

    for i_batch, batch in enumerate(tqdm(val_loader)):
        batch = [i.to(device) for i in batch]
        u_id, i_id, u_attr, i_attr = batch
        
        u_id, i_id, y_true = approx_negative_sampling(u_id, i_id, n_samples=2, bs=batch_size)

        y_pred = model(u_id, i_id)
        loss = criterion(y_pred, y_true)
        
        preds.append(y_pred)
        ground_truths.append(y_true)
        running_loss += loss.item()
        
    pred = torch.cat(preds, dim=0).cpu().numpy()
    ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
    
    test_loss = running_loss / len(val_loader)
    test_score = roc_auc_score(ground_truth, pred)

    return test_loss, test_score

In [ ]:
train_dataset = MFDataset(train_set, data['user_attr'], data['item_attr'])
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, drop_last=True)

val_dataset = MFDataset(valid_set, data['user_attr'], data['item_attr'])
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False, drop_last=True)

In [ ]:
n_users, n_items = user_attr.shape[0], item_attr.shape[0]

model = MF(n_users, n_items, emb_size=32)
model = model.to(device)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
#optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)
optimizer = torch.optim.RMSprop(params=model.parameters(), lr=1e-4, momentum=0.9)

In [ ]:
train(n_epochs=10, print_loss=1000)

In [ ]:
model

In [ ]:
test()

In [ ]:
with torch.no_grad():
    batch_size = train_loader.batch_size
    batch = next(iter(train_loader))
    batch = [i.to(device) for i in batch]
    u_id, i_id, u_attr, i_attr = batch
    
    neg_item_idx = approx_negative_sampling(i_id, bs=batch_size)

    u_id = torch.cat([u_id, u_id])
    i_id = torch.cat([i_id, neg_item_idx])
    y_true = torch.cat([torch.ones(batch_size).to(device),
                        torch.zeros(batch_size).to(device)])

    y_pred = model(u_id, i_id)
    
    print(y_pred.sigmoid(), y_true)

In [ ]:
criterion(y_pred, y_true)

In [ ]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    
def load_model(path):
    model = MF(n_users, n_items, emb_size=32)
    model.load_state_dict(torch.load(path))
    model = model.to(device)
    return model

In [ ]:
#save_model(model, "models/mf_01.pth")

In [ ]:
model = load_model("models/mf_01.pth")

In [ ]:
from reco_env import RecoEnv
from utils import import_data_for_env
import gym

In [ ]:
env = gym.make(RecoEnv.id, **import_data_for_env())

In [ ]:
vc = rec.user_id.value_counts()

In [ ]:
vc

In [ ]:
vc[vc >= 3]

In [ ]:
plt.plot(vc)